In [1]:
import numpy as np
from scipy.stats import unitary_group


We will closely follow the math described by Robert Tucci in "An Introduction to Cartan's KAK Decomposition for QC Programmers"

First, we generate a random desired 4x4 unitary matrix transformation

In [2]:
X = unitary_group.rvs(4)

#Shape of X is 4x4
#Definition of unitary is that hermitian(U) @ U = I 
assert np.allclose(np.conj(X.T) @ X ,np.identity(4))
print("Matrix is unitary.")

Matrix is unitary.


Define some useful matrices

In [3]:
sig_x = np.array([[0,1],[1,0]])
sig_y = np.array([[0,-1j],[-1j,0]])
sig_z = np.array([[1,0],[0,-1]])
sig_1 = np.identity(2)
kron = lambda a,b=None: np.kron(a,b) if b else np.kron(a,a)
sigXX = kron(sig_x)
sigYY = kron(sig_y)
sigZZ = kron(sig_z)

#The Magic Basis
M = (1/np.sqrt(2)) * np.array(
    [[1,0,0,1j],
     [0,1j,1,0],
     [0,1j,-1,0],
     [1,0,0,-1j]]
)
E =  (1/np.sqrt(2)) * np.array(
    [[1,1j,0,0],
     [0,0,1j,1],
     [0,0,1j,-1],
     [1,-1j,0,0j]])



![alt text](image.png)

In [18]:
#phi = lambda A: np.conj(M.T) @ kron(A,np.conj(A)) @ M
#cap_phi = lambda A,B:np.conj(M.T) @ kron(A,np.conj(B)) @ M
X = unitary_group.rvs(4)

#Shape of X is 4x4
#Definition of unitary is that hermitian(U) @ U = I 
assert np.allclose(np.conj(X.T) @ X ,np.identity(4))
print("Matrix is unitary.")
X_prime = np.conj(M.T) @ X @ M

X_R =  np.real(X_prime)
X_I = np.imag(X_prime)
#find the simultaneous diagonalization of X_R and X_I 

Ua,S,Vta = np.linalg.svd(X_R)
Va = np.conj(Vta.T)

B_prime = Ua.T @ X_I @ Va

rankXR = np.linalg.matrix_rank(X_R)
G = B_prime[rankXR:,rankXR:]

evals, P = np.linalg.eig(G)

n = X_R.shape[0]
k = P.shape[0]

Pblock = np.block([
    [P,np.zeros((k, n - k))],
    [np.zeros((n - k, k)),np.identity(n - k)]
])

Q_L = Ua @ Pblock
Q_R = Va @ Pblock

eitheta = Q_L.T @ X_prime @ Q_R
assert np.allclose(eitheta, np.diag(np.diagonal(eitheta)), atol=1e-10)
print("Successfully diagonalized X_prime")


assert np.allclose(Q_L @ eitheta @ Q_R.T, X_prime)
print("Step 2 is complete")

A1tensorA0 = M @ Q_L @ np.conj(M.T)
B1tensorB0 = (M @ Q_R @ np.conj(M.T)).conj().T
eik = M @ eitheta @ np.conj(M.T)

assert np.allclose(A1tensorA0@eik@B1tensorB0, X), "KAK decomposition is incorrect"
print("KAK decomposition is correct")

A = eik
sigx = np.matrix([[0,1],[1,0]])
sigz = np.matrix([[1,0],[0,-1]])
sigy = np.matrix([[0,-1j],[1j,0]])
gamma = lambda u: u @ np.kron(sigy,sigy) @ u.conj().T @ np.kron(sigy,sigy)  #IF WE EXPERIENCE ERRORS, IT MIGHT BE BECAUSE OF AMBIGUITY IN WHETHER TO USE CONJUGATE OR TRANSPOSE

gammaA = gamma(A)
eigvals  = np.linalg.eigvals(gammaA)
eigvals3 = eigvals[:3]

x,y,z = np.angle(eigvals3)

alpha = (x+y)/2
beta = (x+z)/2
delta = (y+z)/2

def Rz(theta):
    return np.array([
        [np.exp(-1j * theta / 2), 0],
        [0, np.exp(1j * theta / 2)]
    ])

# CNOT C₁₂ (control on qubit 0, target on qubit 1)
CN12 = np.array([
    [1, 0, 0, 0],
    [0, 0, 0, 1],
    [0, 0, 1, 0],
    [0, 1, 0, 0]
])


# Build A_canonical using your computed α, β, δ
A_canonical = (CN12 @ 
               np.kron(Rz(delta), Rz(beta)) @ 
               CN12 @ 
               np.kron(np.eye(2), Rz(alpha)))

assert(np.allclose(np.sort(np.angle(eigvals)),np.sort(np.angle(np.linalg.eigvals(gamma(A_canonical)))))), "Eigenvalues of gammaA and gammaA_canonical do not match, alpha beta delta are incorrect"
print("Valid alpha, beta, delta")





Matrix is unitary.
Successfully diagonalized X_prime
Step 2 is complete
KAK decomposition is correct
Valid alpha, beta, delta
